In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1628944772737_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Ler os dados do enem 2019

enem = (
    spark
    .read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .option("delimiter", ";")
    .load("s3://datalake-lucas-igti-edc/raw-data/enem/")
    
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
enem.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- CO_MUNICIPIO_RESIDENCIA: integer (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: integer (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- CO_MUNICIPIO_NASCIMENTO: integer (nullable = true)
 |-- NO_MUNICIPIO_NASCIMENTO: string (nullable = true)
 |-- CO_UF_NASCIMENTO: integer (nullable = true)
 |-- SG_UF_NASCIMENTO: string (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_ESCOLA: integer (nullable = true)
 |-- CO_MU

In [4]:
# Mostrar os seguintes dados do Enem 2019, IDADE, SEXO, ESTADO CIVIL, COR, NACIONALIDADE, NOTA DE CIENCIAS NATURAIS, CIENCIAS HUMANAS, LINGUAGEM E MATEMATICA 
df = (
    enem
    .select("NU_IDADE", "TP_SEXO", "TP_ESTADO_CIVIL", "TP_COR_RACA", "TP_NACIONALIDADE",
           "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Mostra 10 alunos
df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---------------+-----------+----------------+----------+----------+----------+----------+
|NU_IDADE|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|
+--------+-------+---------------+-----------+----------------+----------+----------+----------+----------+
|      36|      M|              1|          3|               1|      null|      null|      null|      null|
|      23|      M|              1|          1|               1|      null|      null|      null|      null|
|      39|      F|              1|          3|               1|      null|      null|      null|      null|
|      25|      F|              1|          1|               1|     483.8|     503.6|     537.3|     392.0|
|      22|      F|              1|          1|               1|     513.6|     575.5|     570.7|     677.0|
|      37|      M|              2|          2|               1|     563.7|     644.9|     564.2|     675.3|
|      22|      F|          

In [6]:
#Mostra quantidade de Alunos que fizeram o Enem em 2019.
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5095270

In [7]:
#IMPORTAR ALGUMAS FUNÇÕES
from pyspark.sql.functions import mean, max, min, col, count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
(
    df
    .groupby("TP_SEXO")
    .agg(
        #media da coluna NU_IDADE com nome de "MED_IDADE"
        mean(col("NU_IDADE")).alias("MED_IDADE"),
        #contagem de pessoa de cada sexo com nome de "CONTAGEM"
        count(col("TP_SEXO")).alias("CONTAGEM"),
        #Nota Máxima em Matemática com o nome de "MAX_NOTA_MT"
        max(col("NU_NOTA_MT")).alias("MAX_NOTA_MT"),
        #Nota Mínima em Matemática com o nome de "MIN_NOTA_MT"
        min(col("NU_NOTA_MT")).alias("MIN_NOTA_MT"),
     )
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+--------+-----------+-----------+
|TP_SEXO|         MED_IDADE|CONTAGEM|MAX_NOTA_MT|MIN_NOTA_MT|
+-------+------------------+--------+-----------+-----------+
|      M|22.179410259492027| 2063448|      985.5|        0.0|
|      F|22.025187142228095| 3031822|      985.0|        0.0|
+-------+------------------+--------+-----------+-----------+

In [ ]:
#Otimizar o Código utilizando formato Parquet

(
    enem
    .write
    .mode("overwrite")
    .format("parquet")
    .partitionBy("year")
    .save("s3://datalake-lucas-igti-edc/staging/enem")
    
)

In [9]:
#Pausar sessão do Spark
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…